## Linking in Spark


<a target="_blank" href="https://colab.research.google.com/github/moj-analytical-services/splink/blob/splink4_examples_notebooks/docs/demos/examples/spark/deduplicate_1k_synthetic.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install git+https://github.com/moj-analytical-services/splink.git@splink4_examples_notebooks
# !pip install pyspark

In [2]:
from splink.spark.jar_location import similarity_jar_location

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import types

conf = SparkConf()
# This parallelism setting is only suitable for a small toy example
conf.set("spark.driver.memory", "12g")
conf.set("spark.default.parallelism", "16")


# Add custom similarity functions, which are bundled with Splink
# documented here: https://github.com/moj-analytical-services/splink_scalaudfs
path = similarity_jar_location()
conf.set("spark.jars", path)

sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession(sc)
spark.sparkContext.setCheckpointDir("./tmp_checkpoints")

24/02/13 16:46:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
# Disable warnings for pyspark - you don't need to include this
import warnings

spark.sparkContext.setLogLevel("ERROR")
warnings.simplefilter("ignore", UserWarning)

In [8]:
from splink.datasets import splink_datasets

pandas_df = splink_datasets.fake_1000

df = spark.createDataFrame(pandas_df)

In [9]:
import splink.comparison_library as cl
import splink.comparison_template_library as ctl
from splink.blocking_rule_library import block_on
from splink.settings_creator import SettingsCreator

settings = SettingsCreator(
    link_type="dedupe_only",
    comparisons=[
        ctl.NameComparison("first_name"),
        ctl.NameComparison("surname"),
        # ctl.date_comparison("dob", cast_strings_to_date=True),
        # TOD=Fix date comparison
        ctl.DateComparison(
            "dob",
            input_is_string=True,
            datetime_metrics=["month", "year", "year"],
            datetime_thresholds=[1, 1, 10],
            datetime_format="%Y%m%d",
        ),
        cl.ExactMatch("city").configure(term_frequency_adjustments=True),
        ctl.EmailComparison("email", include_username_fuzzy_level=False),
    ],
    blocking_rules_to_generate_predictions=[
        block_on("first_name"),
        "l.surname = r.surname",  # alternatively, you can write BRs in their SQL form
    ],
    retain_intermediate_calculation_columns=True,
    em_convergence=0.01,
)

In [11]:
from splink.linker import Linker
from splink.spark.database_api import DuckDBAPI

linker = Linker(df, settings, database_api=SparkAPI(spark_session=spark))
deterministic_rules = [
    "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    "l.email = r.email",
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.6)

Probability two random records match is estimated to be  0.0806.                
This means that amongst all possible pairwise record comparisons, one in 12.41 are expected to match.  With 499,500 total possible comparisons, we expect a total of around 40,246.67 matching pairs


In [12]:
linker.estimate_u_using_random_sampling(max_pairs=5e5)

----- Estimating u probabilities using random sampling -----
                                                                                
Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).


In [13]:
training_blocking_rule = "l.first_name = r.first_name and l.surname = r.surname"
training_session_fname_sname = (
    linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)
)

training_blocking_rule = "l.dob = r.dob"
training_session_dob = linker.estimate_parameters_using_expectation_maximisation(
    training_blocking_rule
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.first_name = r.first_name and l.surname = r.surname

Parameter estimates will be made for the following comparison(s):
    - dob
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name
    - surname


                                                                                
Iteration 1: Largest change in params was -0.697 in probability_two_random_records_match
Iteration 2: Largest change in params was 0.0594 in the m_probability of email, level `All other comparisons`
Iteration 3: Largest change in params was 0.0201 in the m_probability of email, level `All other comparisons`
Iteration 4: Largest change in params was 0.00733 in the m_probability of email, level `All other comparisons`

EM converged after 4 iterations

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).

----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.dob = r.dob

Parameter estimates will be made for the following comparison(s):
    - first_name
    - surname
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are

In [14]:
results = linker.predict(threshold_match_probability=0.9)

In [15]:
results.as_pandas_dataframe(limit=5)

,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,bf_first_name,surname_l,surname_r,...,gamma_city,tf_city_l,tf_city_r,bf_city,bf_tf_adj_city,email_l,email_r,gamma_email,bf_email,match_key
0,16.438427,0.999989,552,553,Riley,Riley,3,11.445406,Kiing,King,...,0,0.016,0.187,0.624766,1.000000,riley.k@zavala.com,riley.k@zavala.com,3,8.414103,0
1,8.580023,0.997394,415,418,Brown,Brown,3,11.445406,Florence,Florence,...,0,0.173,0.001,0.624766,1.000000,fb@reese.com,f@b@reese.com,1,252.564307,0
2,11.468454,0.999647,129,131,Matilda,Matilda,3,11.445406,Barker,NaN,...,1,0.012,0.012,5.890519,5.938272,m.b@bell-brown.com,m.b@bell-brown.com,3,8.414103,0
3,14.659426,0.999961,244,245,Joshua,Joshua,3,11.445406,Williams,NaN,...,1,0.014,0.014,5.890519,5.089947,j.williams@levine-johnson.com,j.williams@levine-johnson.com,3,8.414103,0
4,21.093968,1.000000,196,198,Arthur,Arthur,3,11.445406,Both,Booth,...,0,0.017,0.187,0.624766,1.000000,arthurbooth25@romero-onovan.com,arthurbooth25@romero-donovan.com,2,212.063153,0
